### Get data from quandl

In [60]:
import quandl
import pandas as pd
import numpy as np

Show the data

In [79]:
quandl.api_config = "BxM9-HUoVqZqzxKwwWm1"
df = pd.read_csv('Googl.csv')
df = df.astype({"Date": datetime }, error='ignore')
df.set_index('Date', inplace=True)
df.columns

TypeError: data type not understood

### Keep important columns

In [62]:
df = df[['Adj. Open','Adj. High','Adj. Low','Adj. Close','Adj. Volume',]]

In [63]:
# df.to_csv('Googl.csv', sep=',', encoding='utf-8')  # Save the data to local

### Add percent volatility/percent change

In [64]:
df['HL_PCT'] = (df['Adj. High'] - df['Adj. Close']) / df['Adj. Close'] * 100.0

In [65]:
df['PCT_change'] = (df['Adj. Close'] - df['Adj. Open']) / df['Adj. Open'] * 100.0 
# Daily percent change
df = df[['Adj. Close','HL_PCT','PCT_change','Adj. Volume']] # Columns we care about
df.head()

,Adj. Close,HL_PCT,PCT_change,Adj. Volume
Date,,,,
2004-08-19,50.322842,3.712563,0.324968,44659000.0
2004-08-20,54.322689,0.710922,7.227007,22834300.0
2004-08-23,54.869377,3.729433,-1.227880,18256100.0
2004-08-24,52.597363,6.417469,-5.726357,15247300.0
2004-08-25,53.164113,1.886792,1.183658,9188600.0


In [66]:
forecast_col = 'Adj. Close'
df.fillna(-99999, inplace=True)  # Fill NaN value with -99.999

In [67]:
import math
forecast_out = int(math.ceil(0.01 * len(df)))

In [68]:
forecast_out

35

Create label column

In [69]:
df['label'] = df[forecast_col].shift(-forecast_out)  # Shift label column up *forecast_out days

In [70]:
# df.dropna(inplace=True)
df.head()

,Adj. Close,HL_PCT,PCT_change,Adj. Volume,label
Date,,,,,
2004-08-19,50.322842,3.712563,0.324968,44659000.0,69.078238
2004-08-20,54.322689,0.710922,7.227007,22834300.0,67.839414
2004-08-23,54.869377,3.729433,-1.227880,18256100.0,68.912727
2004-08-24,52.597363,6.417469,-5.726357,15247300.0,70.668146
2004-08-25,53.164113,1.886792,1.183658,9188600.0,71.219849


### Import scikit models

In [71]:
from sklearn import preprocessing, model_selection, svm
from sklearn.linear_model import LinearRegression

### Using model

In [72]:
X = np.array(df.drop(['label'],1))

X = preprocessing.scale(X)
df.dropna(inplace=True)
y = np.array(df['label'])
X_lately = X[-forecast_out:]

X = X[:-forecast_out]


len(X), len(y), len(df)

(3389, 3389, 3389)

In [73]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.2)

In [74]:
# Create classifier using LinearRegression
clf = LinearRegression(n_jobs=-1)
clf.fit(X_train,y_train)
accuracy = clf.score(X_test,y_test)
accuracy

0.9771614000207901

### Predict y

In [75]:
forecast_set = clf.predict(X_lately)
print(forecast_set)
print(accuracy)
print(forecast_out)

[1104.9349484  1074.25107461 1021.4829323  1063.03910529 1074.73991045
 1074.97609462 1093.8507718  1112.86790477 1116.26415778 1124.50667783
 1133.96997717 1130.88721246 1150.32454568 1166.04412472 1137.62395014
 1123.9327441  1089.75768835 1105.21799975 1116.0836767  1122.62572921
 1136.92842972 1151.43764431 1183.15658024 1187.53839339 1159.49229331
 1170.41205237 1172.22066337 1154.86402091 1120.16622315 1116.59965877
 1114.5988115  1071.69389605 1044.26627865 1074.54194675 1022.96234545]
0.9771614000207901
35


### Graphing the data

In [76]:
df.head()

,Adj. Close,HL_PCT,PCT_change,Adj. Volume,label
Date,,,,,
2004-08-19,50.322842,3.712563,0.324968,44659000.0,69.078238
2004-08-20,54.322689,0.710922,7.227007,22834300.0,67.839414
2004-08-23,54.869377,3.729433,-1.227880,18256100.0,68.912727
2004-08-24,52.597363,6.417469,-5.726357,15247300.0,70.668146
2004-08-25,53.164113,1.886792,1.183658,9188600.0,71.219849


In [77]:
import datetime
import matplotlib.pyplot as plt
from matplotlib import style
style = 'ggplot'

# Create empty forecast column
df['Forecast'] = np.nan

# Create datetime
last_date = df.iloc[-1].name
last_unix = last_date.timestamp()
one_day = 86400
next_unix = last_unix + one_day

AttributeError: 'str' object has no attribute 'timestamp'